In [ ]:
"""
%pip install transformers
%pip install datasets
%pip install torch
%pip install tokenizers
"""

'\npip install transformers\npip install datasets\npip install torch\npip install tokenizers\n'

Importamos la lista de moléculas de GitHub y mostramos su tamaño y 5 ejemplos

In [13]:
import urllib.request

url = "https://raw.githubusercontent.com/aspuru-guzik-group/selfies/refs/heads/master/examples/vae_example/datasets/dataJ_250k_rndm_zinc_drugs_clean.txt"

with urllib.request.urlopen(url) as response:
    data = response.read().decode('utf-8')
    Corpus = data.strip().split('\n')

print(f"Se cargaron {len(Corpus)} moléculas.")
print("Primeras 5 moléculas:", Corpus[:5])

Se cargaron 249456 moléculas.
Primeras 5 moléculas: ['CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1', 'C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1', 'N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)cc2)cc1', 'CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c2CCCCC3)C1', 'N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#N)C12CCCCC2']


In [14]:
Corpus[:5]

['CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1',
 'C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1',
 'N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)cc2)cc1',
 'CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c2CCCCC3)C1',
 'N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#N)C12CCCCC2']

In [15]:
#Muestra = [EOS + "CC(=O)O" + BOS, "C1=CC=CC=C1", "C(C(=O)O)N"]

Agregamos el Begin y End of Sequence a cada molécula del Corpus

In [16]:
EOS = "[EOS]"
BOS = "[BOS]"
num = 0
for i in Corpus:
    Corpus[num] = BOS + i + EOS
    num += 1

In [17]:
Corpus[:5]

['[BOS]CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1[EOS]',
 '[BOS]C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1[EOS]',
 '[BOS]N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)cc2)cc1[EOS]',
 '[BOS]CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c2CCCCC3)C1[EOS]',
 '[BOS]N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#N)C12CCCCC2[EOS]']

In [18]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers

Obtenemos el vocabulario con Bert pre entrenado de HuggingFace y lo guardamos en "smiles_tokenizer.json"

In [19]:
tokenizer = Tokenizer(models.BPE()) #Llamar al tokenizer BPE

tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False) #Usamos el protocolo UTF-8 de ByteLevel y sacamos los espacios

trainer = trainers.BpeTrainer(vocab_size=100, special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "[EOS]", "[BOS]"]) #Agregamos el tokenizer con BpeTrainer y agregamos los tokens especiales

tokenizer.train_from_iterator(Corpus, trainer=trainer) #Entrenamos el tokenizer con el Corpus

tokenizer.save("smiles_tokenizer.json")

Aplicamos el tokenizer en una una molecula con la funcion Tokenizar() y muestra la molécula en Smiles, dividida en tokens y con los ids de las tokens

In [20]:
# Tokenizamos una molécula (ejemplo)
def Tokenizar(mol):
    output = tokenizer.encode(mol)

    print("SMILES:", mol)
    print("Tokens:", output.tokens)
    print("IDs:", output.ids)

Tokenizar(Corpus[0])

SMILES: [BOS]CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1[EOS]
Tokens: ['[BOS]', 'CC', '(', 'C', ')(', 'C', ')', 'c', '1', 'ccc', '2', 'o', 'cc', '(', 'CC', '(=', 'O', ')', 'Nc', '3', 'ccccc', '3', 'F', ')', 'c', '2', 'c', '1', '[EOS]']
IDs: [6, 43, 8, 24, 65, 24, 9, 36, 13, 46, 14, 39, 42, 8, 43, 45, 30, 9, 61, 15, 53, 15, 26, 9, 36, 14, 36, 13, 5]
